In [ ]:
#note this doesnt work when first letter isnt the element name as is true for cofactors metals and oddballl atoms, so just grab it from the pdb next time you edit this.!!!!!####################################
import time

# get the current time in seconds since the epoch

import pdbfixer
from openmm.app import PDBFile
import sys
from egnn_pytorch import EGNN_Network
import torch.nn as nn
import numpy as np
import random
import torch
from matplotlib import pyplot as plt
import glob
from collections import defaultdict, OrderedDict
import dask.dataframe as dd
import math
anions = {"HIS":("ND1", "ND2"),
        "ASP":("OD1","OD2"),
        "LYS":("NZ"),
        "TYR":("OH"),
        "GLU":("OE1", "OE2"),
        "CYS":("SG")}

from sklearn.neighbors import NearestNeighbors


elements = {
    "H": 1, "He": 2, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, "F": 9, "Ne": 10,
    "Na": 11, "Mg": 12, "Al": 13, "Si": 14, "P": 15, "S": 16, "Cl": 17, "Ar": 18, "K": 19,
    "Ca": 20, "Sc": 21, "Ti": 22, "V": 23, "Cr": 24, "Mn": 25, "Fe": 26, "Co": 27, "Ni": 28,
    "Cu": 29, "Zn": 30, "Ga": 31, "Ge": 32, "As": 33, "Se": 34, "Br": 35, "Kr": 36, "Rb": 37,
    "Sr": 38, "Y": 39, "Zr": 40, "Nb": 41, "Mo": 42, "Tc": 43, "Ru": 44, "Rh": 45, "Pd": 46,
    "Ag": 47, "Cd": 48, "In": 49, "Sn": 50, "Sb": 51, "I": 53, "Xe": 54, "Cs": 55, "Ba": 56,
    "La": 57, "Ce": 58, "Pr": 59, "Nd": 60, "Pm": 61, "Sm": 62, "Eu": 63, "Gd": 64, "Tb": 65,
    "Dy": 66, "Ho": 67, "Er": 68, "Tm": 69, "Yb": 70, "Lu": 71, "Hf": 72, "Ta": 73, "W": 74,
    "Re": 75, "Os": 76, "Ir": 77, "Pt": 78, "Au": 79, "Hg": 80, "Tl": 81, "Pb": 82, "Bi": 83,
    "Po": 84, "At": 85, "Rn": 86, "Fr": 87, "Ra": 88, "Ac": 89, "Th": 90, "Pa": 91, "U": 92,
    "Np": 93, "Pu": 94, "Am": 95, "Cm": 96, "Bk": 97, "Cf": 98, "Es": 99, "Fm": 100, "Md": 101,
    "No": 102, "Lr": 103, "Rf": 104, "Db": 105, "Sg": 106, "Bh": 107, "Hs": 108, "Mt": 109,
    "Ds": 110, "Rg": 111, "Cn": 112, "Nh": 113, "Fl": 114, "Mc": 115, "Lv": 116, "Ts": 117,
    "Og": 118
}


class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float) *
                             (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        if embed_dim % 2 == 1:
            # Handle odd dimensions by filling the remaining column with cos()
            pe[:, 1::2] = torch.cos(position * div_term)[:, :pe[:, 1::2].shape[1]]
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)  # (max_len, 1, embed_dim)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: Tensor with shape (seq_length, batch_size, embed_dim)
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
class SimpleMultiheadAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(SimpleMultiheadAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
    
    def forward(self, x):
        """
        x: Tensor of shape (seq_length, batch_size, embed_dim)
        """
        attn_outputs, attn_weights = self.multihead_attn(x, x, x)
        return attn_outputs


def EGNN1():
    """This layer creates an embedding as well as hopefully navigates dimerrors produced by putting molecules with
    varying node size.
    in
    feats = 1 x n_atoms

    input can be minimum one atom, this accomdates for rogue solo cofactors and unhydrogonated water."""
    net = EGNN_Network(
        num_positions = 1000, # unless what you are passing in is an unordered set, set this to the maximum sequence length
        dim = 1,
        depth=1,
        num_tokens=118,
        num_nearest_neighbors=1,
        dropout=0.05)
    return net


net = EGNN1()

#A = PositionalEncoding(1)
#mha = SimpleMultiheadAttention(1,1)
optimizer2 = torch.optim.Adam(list(net.parameters()), lr=.009, weight_decay=0.02)
criterion = nn.HuberLoss()
losses=[]
preds=[]
Ts=[]
counter=0
#print(net, (time.time() - to)/60)



class prepper():
    """TODO
    init: put pdbs with read csv file from polars"""
    def __init__(self):
        self.db = "/Users/jessihoernschemeyer/pKaSchNet/pkas.csv" #('/content/drive/MyDrive/pkas.csv') #3directory
        #self.pdbs = self.read_pdbs_from_pkpdb()

    def get_targets(self, calculate=False):
        """TODO: remove, put get_pdbs in init.
        pdbs is the list which i have a data onject for"""
        all_targets=OrderedDict()
        df = dd.read_csv(self.db,delimiter=";").compute().sort_values(["idcode"])
        _pdbs = list(list(df[["idcode"]].to_dict().values())[0].values())
        _pdbs=list(set(_pdbs))
        _pdbs.sort()
        self.pdbs=_pdbs[:1499]

    def fix_pdb(self,pdb):
      """puts amber atom names and resis (needed for any parsing), fixes typesetter errors (need), and messes up the numbering"""
      fixer =pdbfixer.PDBFixer(f"{pdb}.pdb")
      fixer.findNonstandardResidues()
      fixer.replaceNonstandardResidues()
      PDBFile.writeFile(fixer.topology, fixer.positions, f"{pdb}f.pdb")

    def load_pdb(self, pdb):
        """loads everything from pdb name""" 
        try:
            #!wget https://files.rcsb.org/download/{pdb}.pdb
            #self.fix_pdb(pdb)
            counter = 0
            with open(pdb + "f.pdb", "r") as f:
                resis, counter = OrderedDict(), 0
                for line in f:
                    L = line.split()
                    if line.startswith("ATOM"):
                        resname, atomname = L[3], L[2]
                        if resname in anions.keys():
                            resname = resname + str(L[5]) + L[4]
                            if resname in resis:
                                resis[resname][f"{atomname} {counter}"] = (float(L[6]), float(L[7]), float(L[8]))
                                counter += 1
                            else:
                                resis[resname] = {f"{atomname} {counter}": (float(L[6]), float(L[7]), float(L[8]))}
                                counter += 1
        except:
            print(pdb,"fail")
            

        return resis

    def get_pdbs_list(self):
        data ={}
        for pdb in self.pdbs:
            data[pdb] = self.load_pdb(pdb)
        self.residues = data
        return

    def get_data(self):
        """returns dictionary of data with pdbids as keys and residueid indexed positions, atomic node and species information, input features (hydrogen indices), and the number of atoms in each sample (residue).
        all_species: {199l: {ASP1A : {CA:1} , ..., {TYR100 : {OXT:1032} } """
        all_pos, all_species = OrderedDict(),OrderedDict()
        counter=0
        data = self.residues
        for pdb in self.pdbs:
            resis = data[pdb]
        #####residues##########
            species, R =OrderedDict(), OrderedDict()
            for resid, vals in resis.items():
                if resid[:3] in anions.keys():
                    counter, resi_pos, resi_species = 0,OrderedDict(), OrderedDict()
                    for atom, pos in vals.items():
                        node = atom.split()
                        resi_pos[counter] = pos
                        resi_species[node[0]] = counter
                        counter +=1
                    species[resid] = resi_species
                    R[resid] = resi_pos
                else:
                    print(32323)
            all_pos[pdb]= R
            all_species[pdb] = species
        all_rdata = defaultdict(dict)
        for title, d in zip(["Z", "R"], (all_species, all_pos)):
            for key, value in d.items():
                all_rdata[key][title] = value
        self.residues = all_rdata

    def run(self):
        "run data prep"
        self.get_targets(calculate=False)
        self.get_pdbs_list()
        self.get_data()
        
        return self
        
def get_Zs(data):
    """."""
    rdata = data.residues
    all_atomic_numbers = OrderedDict()
    for pdb in data.pdbs:
        zs=OrderedDict()
        for res in list(rdata[pdb]["Z"].keys()):
            try:
                zs[res] = [torch.tensor(elements[a[0]]) for a in list(rdata[pdb]["Z"][res].keys())]
            except:
                print(pdb,"zs")
                continue
        all_atomic_numbers[pdb] = zs
    return all_atomic_numbers


def tar():
    all_targets={}
    with open("/Users/jessihoernschemeyer/pKaSchNet/targets_100k","r") as f:
        rdata={}
        for line in f:
            h=line.split()
            if not h:
                all_targets[pdb]=rdata
                continue
            elif len(h) == 1:
                rdata={}
                pdb=h[0]
                continue
            rdata[h[0]] = torch.tensor(float(h[1]))
    return all_targets

to=time.time()
data = prepper().run()
seconds = time.time() - to 
print(seconds/60)


#all_targets=tar() 
print("data prep done", ((time.time() - to)/60))




def EGNN1():
    """"""
    net = EGNN_Network(
        num_positions = 1000, # unless what you are passing in is an unordered set, set this to the maximum sequence length
        dim = 1,
        depth=1,
        num_tokens=118,
        num_nearest_neighbors=1,
        dropout=0.05)
    return net



refpk = {
    "ASP": 3.79,
    "CYS": 8.67,
    "GLU": 4.20,
    "HIS": 6.74,
    "LYS": 10.46,
    "TYR": 9.59,
}


print((time.time() - to )/60)
  
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors
pdbs,zs=data.pdbs,get_Zs(data)






# Initialize output dictionaries
hoodos = {}
all_hoods = {}
all_nodes,all_conn = {},{}

cutoff_radius = 2

cutoff_radius=4
all_hoods={}
for pdb in pdbs:
    if not zs[pdb]:
        print(pdb)
        continue

    residues_data = data.residues[pdb]
    residue_keys = list(zs[pdb].keys())

    # --- Build lists of tensors and record slice ranges ---
    # Each residue may have a different number of nodes.
    # We'll build two lists: one for coordinates ("R") and one for zs features.
    res_R_list = {}  # list of tensors of shape [n_i, 3]
    res_z_list = {} # list of tensors for zs features (shape may vary)
    # This dict maps each residue key to the slice (start, end) in the concatenated tensors.
    #residue_idx_ranges = {}
    global_start = 0
    keys={}
    for i,key in enumerate(residue_keys):
        # Convert coordinates for this residue to a tensor.
        # Note: using tuple(...) ensures a 2D tensor if the values are already arranged properly.
        #R_tensor = torch.tensor(tuple(residues_data["R"][key].values()))

        res_R_list[i] = torch.tensor(tuple(residues_data["R"][key].values()))#.append(R_tensor)

        # Convert zs data for this residue to a tensor.
        z_tensor = torch.tensor(zs[pdb][key])
        res_z_list[i]=z_tensor
        keys[i] = key

        # Record the index range for this residue in the concatenated tensors.
        n_nodes =  z_tensor.shape[0]
        #residue_idx_ranges[i] = (global_start, global_start + z_tensor.shape[0])
        global_start += z_tensor.shape[0]#n_nodes

    # Concatenate all residues together.
    all_coords = torch.cat(tuple(res_R_list.values()), dim=0).detach().numpy()#torch.cat(res_R_list, dim=0)  # shape: [total_nodes, 3]
    node_features = torch.cat(tuple(res_z_list.values()), dim=0) # shape: [total_nodes, feature_dim] (if feature_dim exists)

    # Optionally, store the node features for the pdb (if needed later).
    #all_nodes[pdb] = node_features

    # --- Build query points for nearest neighbors ---
    # For example, using the last coordinate of each residue.
    #query_points = torch.stack([R[-1] for R in res_R_list])

    # Build a NearestNeighbors model using all_coords.
    nbrs = NearestNeighbors(radius=cutoff_radius).fit(all_coords)
    neigh_indices_list = nbrs.radius_neighbors(torch.stack([R[-1] for R in res_R_list.values()]).detach().numpy(), return_distance=False)

    # --- Build neighborhoods while preserving parent (residue) id ---
    # Instead of using an integer index, we use the residue key as the dictionary key.
    #neighborhoods = {}
    last=0
    Rs,neighbors={},{}
    nmap={}
    # Here, each query corresponds to a residue in the order of residue_keys.
    for i,n in enumerate(neigh_indices_list):
    #contacts = n[1] #connectivity
# for i, neighbor_idx in enumerate(neigh_indices_list):
    # Extract features from the zs tensor for these neighbor indices.
    #hf,hpos = node_features[torch.tensor(contacts)], all_coords[torch.tensor(r[1])]
    # Use the corresponding residue key as the neighborhood key.
    #key = residue_keys[i]
    #neighborhoods[i] = (hf, hpos)
        l= res_z_list[i].shape[0]
        

        
        #for nnodes in n[1:]:
        next_last=last+l
        for j in range(last,next_last):
            nmap[j] = i #n[0] #id map
        
        #Rs[i] = r
        #conn={i:{contacts[0]:r}} #who hes connected to
        last=next_last
        
    #print(nmap)


    hoods={}
    for i in range(len(neigh_indices_list)):
        hood = neigh_indices_list[i]
        
        myi=nmap[hood[0]]
        hoodo = [(res_z_list[nmap[myi]].unsqueeze(1), res_R_list[nmap[myi]])]
        fertig=[myi]
            #print(idx)
        for k in hood[1:]:
            if nmap[k] not in fertig:
                #print(nmap[k])
                
                fertig.append(nmap[k])
                #Set([res_z_list[nmap[k]] for k in hood])
                #print(len(res_z_list[nmap[k]]),len(res_R_list[nmap[k]]))
                
                hoodo.append((res_z_list[nmap[k]].unsqueeze(1), res_R_list[nmap[k]]))
            
            
                #print("else")

        
        #hf, hpos = torch.cat([res_z_list[nmap[k]] for k in hood]), torch.vstack([res_R_list[nmap[k]] for k in hood])#in order
        hoods[i]=torch.vstack([torch.hstack(h) for h in hoodo])
        #print(hf)
        #break

        #hf,hpos = node_features[torch.tensor(hood[1:])], all_coords[torch.tensor(hood[1:])]
        #print(hf,hpos)
        

    #for i,hood in enumerate(conn.values()):

    #r=res_R_list[i]


    all_hoods[pdb] = hoods



In [ ]:
cutoff_radius=10
all_hoods={}
for pdb in pdbs:
    if not zs[pdb]:
        print(pdb)
        continue

    residues_data = data.residues[pdb]
    residue_keys = list(zs[pdb].keys())

    # --- Build lists of tensors and record slice ranges ---
    # Each residue may have a different number of nodes.
    # We'll build two lists: one for coordinates ("R") and one for zs features.
    res_R_list = {}  # list of tensors of shape [n_i, 3]
    res_z_list = {} # list of tensors for zs features (shape may vary)
    # This dict maps each residue key to the slice (start, end) in the concatenated tensors.
    #residue_idx_ranges = {}
    global_start = 0
    keys={}
    for i,key in enumerate(residue_keys):
        # Convert coordinates for this residue to a tensor.
        # Note: using tuple(...) ensures a 2D tensor if the values are already arranged properly.
        #R_tensor = torch.tensor(tuple(residues_data["R"][key].values()))

        res_R_list[i] = torch.tensor(tuple(residues_data["R"][key].values()))#.append(R_tensor)

        # Convert zs data for this residue to a tensor.
        z_tensor = torch.tensor(zs[pdb][key])
        res_z_list[i]=z_tensor
        keys[i] = key

        # Record the index range for this residue in the concatenated tensors.
        n_nodes =  z_tensor.shape[0]
        #residue_idx_ranges[i] = (global_start, global_start + z_tensor.shape[0])
        global_start += z_tensor.shape[0]#n_nodes

    # Concatenate all residues together.
    all_coords = torch.cat(tuple(res_R_list.values()), dim=0).detach().numpy()#torch.cat(res_R_list, dim=0)  # shape: [total_nodes, 3]
    node_features = torch.cat(tuple(res_z_list.values()), dim=0) # shape: [total_nodes, feature_dim] (if feature_dim exists)

    # Optionally, store the node features for the pdb (if needed later).
    #all_nodes[pdb] = node_features

    # --- Build query points for nearest neighbors ---
    # For example, using the last coordinate of each residue.
    #query_points = torch.stack([R[-1] for R in res_R_list])

    # Build a NearestNeighbors model using all_coords.
    nbrs = NearestNeighbors(radius=cutoff_radius).fit(all_coords)
    neigh_indices_list = nbrs.radius_neighbors(torch.stack([R[-1] for R in res_R_list.values()]).detach().numpy(), return_distance=False)

    # --- Build neighborhoods while preserving parent (residue) id ---
    # Instead of using an integer index, we use the residue key as the dictionary key.
    #neighborhoods = {}
    last=0
    Rs,neighbors={},{}
    nmap={}
    # Here, each query corresponds to a residue in the order of residue_keys.
    for i,n in enumerate(neigh_indices_list):
    #contacts = n[1] #connectivity
# for i, neighbor_idx in enumerate(neigh_indices_list):
    # Extract features from the zs tensor for these neighbor indices.
    #hf,hpos = node_features[torch.tensor(contacts)], all_coords[torch.tensor(r[1])]
    # Use the corresponding residue key as the neighborhood key.
    #key = residue_keys[i]
    #neighborhoods[i] = (hf, hpos)
        l= res_z_list[i].shape[0]
        

        
        #for nnodes in n[1:]:
        next_last=last+l
        for j in range(last,next_last):
            nmap[j] = i #n[0] #id map
        
        #Rs[i] = r
        #conn={i:{contacts[0]:r}} #who hes connected to
        last=next_last
        
    #print(nmap)


    hoods={}
    for i in range(len(neigh_indices_list)):
        hood = neigh_indices_list[i]
        
        myi=nmap[hood[0]]
        hoodo = {myi:(res_z_list[nmap[myi]].unsqueeze(1), res_R_list[nmap[myi]])}
        fertig={myi:myi}
            #print(idx)
        for k in hood[1:]:
            if k not in fertig:
                #print(nmap[k])
                
                fertig[k]=nmap[k]
                #Set([res_z_list[nmap[k]] for k in hood])
                #print(len(res_z_list[nmap[k]]),len(res_R_list[nmap[k]]))
                
                hoodo[k]=(res_z_list[nmap[k]].unsqueeze(1), res_R_list[nmap[k]])
            
            
                #print("else")

        
        #hf, hpos = torch.cat([res_z_list[nmap[k]] for k in hood]), torch.vstack([res_R_list[nmap[k]] for k in hood])#in order
        hoods[i]=torch.vstack([torch.hstack(h) for h in hoodo.values()])
        #print(hf)
        #break

        #hf,hpos = node_features[torch.tensor(hood[1:])], all_coords[torch.tensor(hood[1:])]
        #print(hf,hpos)
        

    #for i,hood in enumerate(conn.values()):

    #r=res_R_list[i]


    all_hoods[pdb] = hoods



In [ ]:

for i,hood in enumerate(all_hoods.values()):
    for j,h in enumerate(hood.values()):
        
        
    
    #input = torch.hstack((h[0][0].unsqueeze(1),h[0][1]))
        torch.save(h,f"./4A/{i}_{j}")

torch.load("./4A/7_3")